<a href="https://colab.research.google.com/github/iiCellxx/CSST-101-ALMARIO/blob/main/PRACTICE-CHATBOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install tensorflow


In [45]:
!pip install tqdm


# 1. Import Libraries

In [30]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint


# 2. Load and Preprocess the Data

In [20]:
# Load your synthetic cadet data from a CSV file
cadet_data = pd.read_csv("synthetic_cadet_data.csv")
cadet_data.head()

,Cadet_ID,Name,Class,Rank,Grade Equivalent,Previous Designation,Behavior
0,C001,Cadet A,1st Class,C/CPT,92.5,S1,Excellent
1,C002,Cadet B,1st Class,C/CPT,87.3,S2,Good
2,C003,Cadet C,2nd Class,C/MAJ,94.2,S3,Excellent
3,C004,Cadet D,1st Class,C/MAJ,88.6,S4,Good
4,C005,Cadet E,2nd Class,C/MAJ,80.5,S7,Excellent


In [31]:
# Load your synthetic cadet data from a CSV file
cadet_data = pd.read_csv("synthetic_cadet_data.csv")

# Convert categorical columns (Class, Rank, Behavior, Designation) to numeric values
cadet_data['Class'] = cadet_data['Class'].map({"1st Class": 1, "2nd Class": 2})
cadet_data['Rank'] = cadet_data['Rank'].map({"C/CPT": 1, "C/MAJ": 2})
cadet_data['Behavior'] = cadet_data['Behavior'].map({"Excellent": 0, "Good": 1, "Bad": 2})
cadet_data['Previous Designation'] = cadet_data['Previous Designation'].map({
    "S1": 0, "S2": 1, "S3": 2, "S4": 3, "S5": 4, "S7": 5
})

# Map Rank to Designation
designation_map = {
    "CORPS COMMANDER": 0, "CORPS EXECUTIVE OFFICER": 1, "CORPS G1": 2, "CORPS G2": 3,
    "CORPS G3": 4, "CORPS G4": 5, "CORPS G7": 6, "CORPS ADJUTANT": 7
}
cadet_data['Designation'] = cadet_data['Rank'].map({1: "CORPS EXECUTIVE OFFICER", 2: "CORPS COMMANDER"})
cadet_data['Designation'] = cadet_data['Designation'].map(designation_map)


# 3. Feature Engineering and Tokenization

In [32]:
# Combine important features into a single string to form the input for the model
cadet_data['Feature'] = cadet_data[['Class', 'Rank', 'Grade Equivalent', 'Previous Designation', 'Behavior']].apply(
    lambda x: ' '.join(str(val) for val in x), axis=1)

# Tokenize the 'Feature' column (input text data)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(cadet_data['Feature'])
X = tokenizer.texts_to_sequences(cadet_data['Feature'])
X = pad_sequences(X, padding='post')


# 4. Prepare the Target Variable

In [33]:
# Prepare the target (Designation) for the model
y = to_categorical(cadet_data['Designation'], num_classes=len(designation_map))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# 5. Define the Model

In [35]:
# Define the LSTM model architecture
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 128, input_length=X.shape[1]))  # Embedding layer
model.add(SpatialDropout1D(0.2))  # Dropout layer to prevent overfitting
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))  # LSTM layer
model.add(Dense(100, activation='relu'))  # Dense layer with ReLU activation
model.add(Dense(len(designation_map), activation='softmax'))  # Output layer (multi-class classification)

# Compile the model with categorical crossentropy loss and Adam optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# 6. Train the Model


In [46]:
from tqdm.keras import TqdmCallback

# Define a ModelCheckpoint callback to save the best model during training
checkpoint_callback = ModelCheckpoint(
    'cadet_designation_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

# Use TqdmCallback for a progress bar
progress_bar_callback = TqdmCallback(verbose=1)

# Train the model with the checkpoint callback and progress bar
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test),
          callbacks=[checkpoint_callback, progress_bar_callback])


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.6250 - loss: 1.6551
Epoch 1: val_accuracy improved from -inf to 0.00000, saving model to cadet_designation_model.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step - accuracy: 0.6250 - loss: 1.6551 - val_accuracy: 0.0000e+00 - val_loss: 1.7787
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.6250 - loss: 1.5292
Epoch 2: val_accuracy did not improve from 0.00000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - accuracy: 0.6250 - loss: 1.5292 - val_accuracy: 0.0000e+00 - val_loss: 1.7181
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.6250 - loss: 1.4458
Epoch 3: val_accuracy did not improve from 0.00000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.6250 - loss: 1.4458 - val_accuracy: 0.0000e+00 - val_loss: 1.6556
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.6250 - loss: 1.2776
Epoch 4: val_accuracy did not improve from 0.00000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.6250 -

In [51]:
from tensorflow.keras.models import load_model

# Load the Keras model
model = load_model('cadet_designation_model.keras')

# Print a summary of the model's architecture
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 10, 128)             │           2,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ spatial_dropout1d_2                  │ (None, 10, 128)             │               0 │
│ (SpatialDropout1D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 100)                 │          91,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 100)                 │          10,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 8)                   │             808 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 314,822 (1.20 MB)

 Trainable params: 104,940 (409.92 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 209,882 (819.86 KB)

# 7. Define Eligibility Functions

In [47]:
# Define eligibility conditions for various designations
def is_eligible_for_corp_commander(cadet):
    return cadet['Class'] in [1, 2] and cadet['Grade Equivalent'] >= 90 and \
           cadet['Previous Designation'] in [0, 1, 2, 3, 6] and \
           cadet['Behavior'] in [0, 1]  # Excellent or Good

def is_eligible_for_corp_executive_officer(cadet):
    return cadet['Class'] in [1, 2] and cadet['Grade Equivalent'] >= 85 and \
           cadet['Previous Designation'] in [0, 1, 2, 3, 6] and \
           cadet['Behavior'] in [0, 1]  # Excellent or Good

def is_eligible_for_corp_g1(cadet):
    return cadet['Class'] in [1, 2] and cadet['Grade Equivalent'] >= 85 and \
           cadet['Previous Designation'] in [0, 6] and \
           cadet['Behavior'] in [0, 1]

def is_eligible_for_corp_g2(cadet):
    return cadet['Class'] in [1, 2] and cadet['Grade Equivalent'] >= 80 and \
           cadet['Previous Designation'] in [0, 2, 6] and \
           cadet['Behavior'] in [0, 1]

def is_eligible_for_corp_g3(cadet):
    return cadet['Class'] in [1, 2] and cadet['Grade Equivalent'] >= 80 and \
           cadet['Previous Designation'] in [0, 3, 6] and \
           cadet['Behavior'] in [0, 1]

def is_eligible_for_corp_g4(cadet):
    return cadet['Class'] in [1, 2] and cadet['Grade Equivalent'] >= 75 and \
           cadet['Previous Designation'] in [0, 4, 6] and \
           cadet['Behavior'] in [0, 1]

def is_eligible_for_corp_g7(cadet):
    return cadet['Class'] in [1, 2] and cadet['Grade Equivalent'] >= 75 and \
           cadet['Previous Designation'] in [0, 5, 6] and \
           cadet['Behavior'] in [0, 1]

def is_eligible_for_corp_adjutant(cadet):
    return cadet['Class'] in [1, 2] and cadet['Grade Equivalent'] >= 80 and \
           cadet['Previous Designation'] in [0, 6] and \
           cadet['Behavior'] in [0, 1]


# 8. Prediction Function

In [48]:
# Function to predict the designation of a cadet based on eligibility
def predict_designation(cadet):
    # Prepare the cadet input by extracting only relevant columns
    # Convert 'Grade Equivalent' to an integer index if it's a categorical feature
    # For example, if you have 19 grade categories, you could map them to indices 0-18:
    # grade_index = int(cadet['Grade Equivalent'])  # Assuming grade is an integer
    # If grade is a continuous value, you might need to discretize it into categories first.

    # Ensure the 'Grade Equivalent' is within the valid range for your embedding layer
    # This might involve clipping or scaling the value if necessary.
    grade_index = min(int(cadet['Grade Equivalent']), 18)

    cadet_input = np.array([[cadet['Class'], cadet['Rank'], grade_index,
                             cadet['Previous Designation'], cadet['Behavior']]])

    # Pad the sequence if necessary (if your model expects padded input)
    cadet_input = pad_sequences(cadet_input, padding='post', maxlen=X.shape[1])

    # Make the prediction
    pred = model.predict(cadet_input)
    designation_idx = np.argmax(pred)  # Find the index of the predicted designation
    designation = list(designation_map.keys())[designation_idx]  # Map the index to the designation name

    # Check eligibility based on predefined rules
    eligible_designations = []
    if is_eligible_for_corp_commander(cadet):
        eligible_designations.append("CORPS COMMANDER")

    # Similar checks for other designations

    # Display the results
    if eligible_designations:
        print(f"{cadet['Name']} is eligible for the following designation(s):")
        for designation in eligible_designations:
            print(f"- {designation}")
    else:
        print(f"{cadet['Name']} does not meet the criteria for any designation.")

In [49]:
# Sample cadet data
cadet_sample = {
    'Name': 'Cadet A',
    'Class': 1,  # 1st Class
    'Rank': 1,  # C/CPT
    'Grade Equivalent': 92.5,
    'Previous Designation': 0,  # S1
    'Behavior': 0  # Excellent
}

# Predict designation
predict_designation(cadet_sample)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Cadet A is eligible for the following designation(s):
- CORPS COMMANDER
